<a href="https://colab.research.google.com/github/Avishkarm/AtopicDermatitisDetection/blob/main/eczema_detection_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This file uses pre-trained model to
1. classify images as eczema or non-eczema
2. determine the level of eczema

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow-hub


In [ ]:
# Clone the GitHub repository
!git clone https://github.com/biswa932/EczemaDetectionUsingCNN.git

# Navigate to the cloned repository
%cd EczemaDetectionUsingCNN

# Verify if the model file is downloaded
!ls


fatal: destination path 'EczemaDetectionUsingCNN' already exists and is not an empty directory.
/content/EczemaDetectionUsingCNN
03DermatitisArm1.jpg		  eczemaLevelDetection.h5
alto.jpg			  EczemaLevelDetectionModelFile.mlmodel
black.jpg			  handClear.jpg
desquamation-4.jpg		  README.md
eczema-acute-4.jpg		  skinDetection.h5
eczema-arms-8.jpg		  SkinDetectionModelFile.mlmodel
eczemaDetection.h5		  ThreeStageEczemaDetection.ipynb
EczemaDetectionModelFile.mlmodel  ThreeStageEczemaDetectionToCoreML.ipynb


# Classifying as eczema or non - eczema

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import load_model

# Define the model path
model_path = "/content/EczemaDetectionUsingCNN/eczemaDetection.h5"

# Load the model without compiling
model = load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer}, compile=False)

# Recompile the model with the correct loss and optimizer
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Display model structure
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1792)              4363712   
                                                                 
 dense_1 (Dense)             (None, 2)                 3586      
                                                                 
Total params: 4,367,298
Trainable params: 3,586
Non-trainable params: 4,363,712
_________________________________________________________________


To prompt user to input a single image and check for eczema.

In [ ]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from google.colab import files

# Function to preprocess the image
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)  # Load image
    img_array = img_to_array(img) / 255.0  # Convert to array and normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Prompt user to upload an image
print("Please upload an image for classification:")
uploaded = files.upload()

# Get the uploaded file's path
for filename in uploaded.keys():
    image_path = filename

# Preprocess the image
input_image = preprocess_image(image_path)

# Make predictions
predictions = model.predict(input_image)

# Interpret the results
class_names = ['Non-Eczema', 'Eczema']
predicted_class = class_names[np.argmax(predictions)]

# Display results
print(f"Predicted Class: {predicted_class}")
print(f"Confidence Scores: {predictions[0]}")


Please upload an image for classification:


Saving eczema.jpg to eczema (1).jpg
1/1 [==============================] - 1s 979ms/step
Predicted Class: Eczema
Confidence Scores: [0.6180905  0.74297094]


To apply the model reccursively to multiple images. (Images have been saved in a folder in google drive)

In [ ]:
import os
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from google.colab import drive

# Function to preprocess the image
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)  # Load image
    img_array = img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Path to the folder in your Google Drive
folder_path = "/content/drive/My Drive/eczema_eyes/unseen_images"  # Replace with your folder path

# Get a list of all files in the folder
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Classify each image in the folder
class_names = ['Non-Eczema', 'Eczema']

for image_path in image_files:
    print(f"Processing image: {os.path.basename(image_path)}")

    # Preprocess the image
    input_image = preprocess_image(image_path)

    # Make predictions
    predictions = model.predict(input_image)

    # Interpret the results
    predicted_class = class_names[np.argmax(predictions)]
    confidence_scores = predictions[0]

    # Display results
    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence Scores: {confidence_scores}")
    print("-" * 50)


Processing image: eczema1.jpg
1/1 [==============================] - 0s 82ms/step
Predicted Class: Eczema
Confidence Scores: [0.6180905  0.74297094]
--------------------------------------------------
Processing image: eczema2.jpg
1/1 [==============================] - 0s 83ms/step
Predicted Class: Non-Eczema
Confidence Scores: [0.8237816  0.41264042]
--------------------------------------------------
Processing image: eczema3.jpg
1/1 [==============================] - 0s 79ms/step
Predicted Class: Eczema
Confidence Scores: [0.38220862 0.63907427]
--------------------------------------------------
Processing image: eczema4.jpg
1/1 [==============================] - 0s 83ms/step
Predicted Class: Eczema
Confidence Scores: [0.51913434 0.55493206]
--------------------------------------------------
Processing image: eczema5.jpg
1/1 [==============================] - 0s 76ms/step
Predicted Class: Non-Eczema
Confidence Scores: [0.8841709  0.17893748]
-------------------------------------------

# Eczema Level Detection

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import load_model

# Define the model path
model_path = "/content/EczemaDetectionUsingCNN/eczemaLevelDetection.h5"

# Load the model without compiling
model = load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer}, compile=False)

# Recompile the model with the correct loss and optimizer
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Display model structure
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1792)              4363712   
                                                                 
 dense_3 (Dense)             (None, 3)                 5379      
                                                                 
Total params: 4,369,091
Trainable params: 5,379
Non-trainable params: 4,363,712
_________________________________________________________________


To prompt user to input a single image and determine the level/severity of eczema.

In [ ]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from google.colab import files

# Function to preprocess the image
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)  # Load image
    img_array = img_to_array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Class labels for eczema levels
class_names = ['Mild', 'Moderate', 'Severe']  # Update labels if different

# Prompt the user to upload an image
print("Please upload an image for eczema level detection:")
uploaded = files.upload()

# Process the uploaded image
for filename in uploaded.keys():
    image_path = filename  # Get the path of the uploaded image
    input_image = preprocess_image(image_path)

    # Make predictions
    predictions = model.predict(input_image)

    # Interpret the results
    predicted_class = class_names[np.argmax(predictions)]
    confidence_scores = predictions[0]

    # Display the results
    print(f"\nImage: {filename}")
    print(f"Predicted Eczema Level: {predicted_class}")
    print(f"Confidence Scores: {confidence_scores}")


Please upload an image for eczema level detection:


Saving normal.jpg to normal (1).jpg
1/1 [==============================] - 0s 80ms/step

Image: normal (1).jpg
Predicted Eczema Level: Mild
Confidence Scores: [0.755553   0.3137466  0.53496367]


To apply the model reccursively to multiple images. (Images have been saved in a folder in google drive)

In [ ]:
import os
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from google.colab import drive

# Function to preprocess the image
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)  # Load image
    img_array = img_to_array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Mount Google Drive
drive.mount('/content/drive')

# Path to the folder in your Google Drive
folder_path = "/content/drive/My Drive/eczema_eyes/unseen_images"  # Replace with your folder path

# Get a list of all image files in the folder
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Class labels for eczema levels (Update based on your model's output classes)
class_names = ['Mild', 'Moderate', 'Severe']  # Update class labels if needed

# Classify each image in the folder
for image_path in image_files:
    print(f"Processing image: {os.path.basename(image_path)}")

    # Preprocess the image
    input_image = preprocess_image(image_path)

    # Make predictions
    predictions = model.predict(input_image)

    # Interpret the results
    predicted_class = class_names[np.argmax(predictions)]
    confidence_scores = predictions[0]

    # Display results
    print(f"Predicted Eczema Level: {predicted_class}")
    print(f"Confidence Scores: {confidence_scores}")
    print("-" * 50)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing image: eczema1.jpg
1/1 [==============================] - 0s 75ms/step
Predicted Eczema Level: Moderate
Confidence Scores: [0.18650259 0.700373   0.2327971 ]
--------------------------------------------------
Processing image: eczema2.jpg
1/1 [==============================] - 0s 75ms/step
Predicted Eczema Level: Moderate
Confidence Scores: [0.61498135 0.7363337  0.01151701]
--------------------------------------------------
Processing image: eczema3.jpg
1/1 [==============================] - 0s 75ms/step
Predicted Eczema Level: Mild
Confidence Scores: [0.63847965 0.52600527 0.25193837]
--------------------------------------------------
Processing image: eczema4.jpg
1/1 [==============================] - 0s 91ms/step
Predicted Eczema Level: Moderate
Confidence Scores: [0.46685794 0.54463387 0.07482943]
----------------------------------------------